# National League Backtest
This notebook simulates wagering strategies on National League matches using model-derived probabilities and available market odds. It tracks bankroll performance, return on investment (ROI), and maximum drawdown and compares the model-based approach against a simple baseline strategy of betting Over 2.5 goals when the odds are below 1.70.

In [1]:

from pathlib import Path
import os
import pandas as pd
import numpy as np

# Ensure we run from the repository root
repo_root = Path().resolve().parents[1]
os.chdir(repo_root)

files = sorted(Path('.').glob('england-national-league-matches-*-stats.csv'))

df = pd.concat(pd.read_csv(f) for f in files)
# Keep relevant columns and drop rows with missing data
cols = ['timestamp','total_goal_count','odds_ft_over25','over_25_percentage_pre_match']
df = df[cols].dropna()
# Convert percentage to probability
df['model_prob'] = df['over_25_percentage_pre_match'] / 100.0
df.sort_values('timestamp', inplace=True)


In [2]:

def simulate(strategy, stake=1.0, starting_bankroll=100.0):
    bankroll = starting_bankroll
    bankroll_history = [bankroll]
    total_staked = 0.0
    for _, row in df.iterrows():
        odds = row['odds_ft_over25']
        p = row['model_prob']
        result = row['total_goal_count'] > 2
        if strategy(p, odds):
            bankroll -= stake
            total_staked += stake
            if result:
                bankroll += stake * odds
        bankroll_history.append(bankroll)
    profit = bankroll - starting_bankroll
    roi = profit / total_staked if total_staked else 0.0
    bankroll_series = pd.Series(bankroll_history)
    running_max = bankroll_series.cummax()
    drawdown = running_max - bankroll_series
    max_drawdown = drawdown.max()
    metrics = {
        'final_bankroll': bankroll,
        'profit': profit,
        'roi': roi,
        'max_drawdown': max_drawdown,
        'total_staked': total_staked,
    }
    return bankroll_history, metrics

def model_strategy(p, odds):
    return p * odds > 1  # positive expected value


def baseline_strategy(p, odds):
    return odds < 1.70

model_history, model_results = simulate(model_strategy)
baseline_history, baseline_results = simulate(baseline_strategy)

print('Model-based strategy:', model_results)
print('Baseline strategy:', baseline_results)


Model-based strategy: {'final_bankroll': np.float64(91.64999999999999), 'profit': np.float64(-8.350000000000009), 'roi': np.float64(-0.023521126760563404), 'max_drawdown': np.float64(26.040000000000063), 'total_staked': 355.0}
Baseline strategy: {'final_bankroll': np.float64(-418.1199999999999), 'profit': np.float64(-518.1199999999999), 'roi': np.float64(-0.5668708971553609), 'max_drawdown': np.float64(529.62), 'total_staked': 914.0}
